In [1]:
n_features = 50
n_timesteps_in = 8
n_timesteps_out = 5

n_hidden_units = 200

pad_value = 0

batch_size = 128

In [2]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
import numpy as np

/home/tony/anaconda3/envs/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(0, n_unique-1) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
    encoding = []
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [4]:
# prepare data for the LSTM
def get_pair(n_in, n_out, cardinality):
    # generate random sequence
    sequence_in = generate_sequence(n_in, cardinality)
    sequence_transform = sequence_in[::-1]  # Reverse sequence
    sequence_out = sequence_transform[:n_out] + [pad_value for _ in range(n_in-n_out)]
    # one hot encode
    X = one_hot_encode(sequence_in, cardinality)
    y = one_hot_encode(sequence_out, cardinality)
    # reshape as 3D
    X = X.reshape((1, X.shape[0], X.shape[1]))
    y = y.reshape((1, y.shape[0], y.shape[1]))
    return X,y

In [5]:
def get_batch(n_in, n_out, cardinality, batch_size):
    
    while True:
        
        X_batch = []
        y_batch = []
        
        for idx in range(batch_size):
            
            X,y = get_pair(n_in, n_out, cardinality)
            X_batch.append(np.array(X[0]))
            y_batch.append(np.array(y[0]))
            
        yield np.array(X_batch), np.array(y_batch)

In [6]:
# define model
model = Sequential()
model.add(LSTM(n_hidden_units, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(n_hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation="softmax")))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [7]:
num_epochs = 7
training_steps = 1000
validation_steps = 100

train_generator = get_batch(n_timesteps_in, n_timesteps_out, n_features, batch_size)
validate_generator = get_batch(n_timesteps_in, n_timesteps_out, n_features, batch_size)

history = model.fit_generator(train_generator, steps_per_epoch=training_steps, epochs=num_epochs, 
                              validation_data=validate_generator, validation_steps=validation_steps,
                              verbose=2)

Epoch 1/7
 - 25s - loss: 1.5353 - acc: 0.5486 - val_loss: 0.6844 - val_acc: 0.7651
Epoch 2/7
 - 25s - loss: 0.2325 - acc: 0.9323 - val_loss: 0.0299 - val_acc: 0.9948
Epoch 3/7
 - 25s - loss: 0.0143 - acc: 0.9975 - val_loss: 0.0068 - val_acc: 0.9989
Epoch 4/7
 - 26s - loss: 0.0037 - acc: 0.9994 - val_loss: 0.0031 - val_acc: 0.9993
Epoch 5/7
 - 25s - loss: 0.0019 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9998
Epoch 6/7
 - 25s - loss: 0.0012 - acc: 0.9997 - val_loss: 0.0015 - val_acc: 0.9996
Epoch 7/7
 - 25s - loss: 9.3717e-04 - acc: 0.9998 - val_loss: 0.0012 - val_acc: 0.9996


In [8]:
# Check a few example predictions to sanity check trained model
num_examples=10
for idx in range(num_examples):
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    yhat = model.predict(X, verbose=0)
    print("*"*20)
    print("Test case #{}".format(idx+1))
    print("Input = ", one_hot_decode(X[0]))
    print("Expected Output:", one_hot_decode(y[0]))
    print("Model Predicted Output:", one_hot_decode(yhat[0]))
    print("*"*20)
    print("\n")

********************
Test case #1
Input =  [9, 27, 44, 15, 12, 8, 24, 12]
Expected Output: [12, 24, 8, 12, 15, 0, 0, 0]
Model Predicted Output: [12, 24, 8, 12, 15, 0, 0, 0]
********************


********************
Test case #2
Input =  [42, 12, 1, 22, 16, 8, 10, 3]
Expected Output: [3, 10, 8, 16, 22, 0, 0, 0]
Model Predicted Output: [3, 10, 8, 16, 22, 0, 0, 0]
********************


********************
Test case #3
Input =  [31, 3, 37, 11, 28, 24, 14, 24]
Expected Output: [24, 14, 24, 28, 11, 0, 0, 0]
Model Predicted Output: [24, 14, 24, 28, 11, 0, 0, 0]
********************


********************
Test case #4
Input =  [34, 22, 3, 21, 28, 20, 45, 17]
Expected Output: [17, 45, 20, 28, 21, 0, 0, 0]
Model Predicted Output: [17, 45, 20, 28, 21, 0, 0, 0]
********************


********************
Test case #5
Input =  [36, 5, 22, 49, 14, 5, 47, 30]
Expected Output: [30, 47, 5, 14, 49, 0, 0, 0]
Model Predicted Output: [30, 47, 5, 14, 49, 0, 0, 0]
********************


*****************